In [1]:
import pandas as pd
import numpy as np

In [19]:
data_path = "C:/Users/minkyu/Desktop/open/"

In [20]:
df = pd.read_csv(data_path+"train.csv" )

In [37]:
df.head(5)

,ID,발생일시,사고인지 시간,날씨,기온,습도,공사종류,연면적,층 정보,인적사고,...,공사종류(대분류),공사종류(중분류),공종(대분류),공종(중분류),사고객체(대분류),사고객체(중분류),월,시간,지상층,지하층
0,TRAIN_00000,2023-12-31 오후 12:44,정규작업,맑음,1℃,30%,건축 / 건축물 / 근린생활시설,"4,892.77㎡","지상 14층, 지하 3층",떨어짐(5미터 이상 ~ 10미터 미만),...,건축,건축물,건축,철근콘크리트공사,건설자재,철근,12,12,14,3
1,TRAIN_00001,2023-12-30 오후 03:35,정규작업,맑음,10℃,90%,토목 / 터널 / 철도터널,NaN,NaN,끼임,...,토목,터널,토목,터널공사,건설자재,볼트,12,15,<NA>,<NA>
2,TRAIN_00002,2023-12-30 오후 02:30,정규작업,맑음,14℃,70%,건축 / 건축물 / 업무시설,"1,994.62㎡","지상 5층, 지하 0층",넘어짐(미끄러짐),...,건축,건축물,건축,철근콘크리트공사,기타,기타,12,14,5,0
3,TRAIN_00003,2023-12-30 오후 12:00,휴일근무,흐림,12℃,55%,토목 / 하천 / 기타,NaN,NaN,기타,...,토목,하천,토목,하천공사,부재,교각 기초,12,12,<NA>,<NA>
4,TRAIN_00004,2023-12-30 오전 10:00,정규작업,맑음,0℃,10%,건축 / 건축물 / 공동주택,"59,388.93㎡","지상 27층, 지하 3층",넘어짐(미끄러짐),...,건축,건축물,건축,해체 및 철거공사,기타,건설폐기물,12,10,27,3


In [22]:
df.replace('-', np.nan, inplace=True)
df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()




In [23]:
# 발생일시 전처리 -> 월과 시간만 빼서 저장 
def extract_month_and_hour(s):
    date_part, period, time_part = s.split()
    month = int(date_part.split('-')[1])
    hour_str, _ = time_part.split(':')
    hour = int(hour_str)
    
    if period == "오후" and hour != 12:
        hour += 12
    elif period == "오전" and hour == 12:
        hour = 0
        
    return pd.Series([month, hour])
df[['월', '시간']] = df['발생일시'].apply(extract_month_and_hour)


In [24]:
# 지하, 지상으로 분리 
df['지상층'] = df['층 정보'].str.extract(r'지상\s*(\d+)층')[0].astype('Int64')
df['지하층'] = df['층 정보'].str.extract(r'지하\s*(\d+)층')[0].astype('Int64')


In [25]:
df['사고인지 시간'].value_counts()

사고인지 시간
정규작업       17811
기타           688
휴일근무         623
작업 전         524
식사ㆍ휴식 등      273
출ㆍ퇴근 등       160
불류불능         114
연장근무         103
Name: count, dtype: int64

In [27]:
df.isna().sum()

ID                    0
발생일시                  0
사고인지 시간            3126
날씨                    0
기온                    0
습도                    0
공사종류                  0
연면적                8088
층 정보               8200
인적사고                 32
물적사고               1490
공종                   11
사고객체                687
작업프로세스               63
장소                    0
부위                    0
사고원인                 67
재발방지대책 및 향후조치계획       0
공사종류(대분류)             0
공사종류(중분류)             1
공종(대분류)              11
공종(중분류)              11
사고객체(대분류)           687
사고객체(중분류)           687
월                     0
시간                    0
지상층                8200
지하층                8200
dtype: int64

In [41]:
df.to_csv(data_path + 'df.csv', encoding='utf-8-sig', index=False)


In [ ]:
# 훈련 데이터 통합 생성
combined_training_data = df.apply(
    lambda row: {
        "question": (
            f"{row['월']}월 , 날씨 {row['날씨']} ,기온 , 습도, " 
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)